## Matching Stability Experiments

We consider first an example and check how stable is the matching.

First of all, we generate a distribution around three circles together with uniform noise on the background. For this we compute cycle affinities as in the article "Cycle Registration in Persistent Homology with Applications in Topological Bootstrap". All samples are drawn from the same distribution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.spatial.distance as dist

import IBloFunMatch_inter as ibfm
output_dir = "output" 

In [ ]:
def three_circles_distribution(p, n, RandGen):
    choice_C_or_U = list(RandGen.binomial(n=1, p=p, size=n))
    num_uniform = choice_C_or_U.count(0)
    uniform = np.vstack((RandGen.uniform(-5,5, num_uniform), RandGen.uniform(-3,6, num_uniform))).transpose()
    choice_C = list(RandGen.choice(3, choice_C_or_U.count(1)))
    C0 = ibfm.sampled_circle(2,3,choice_C.count(0), RandGen)+[2,0]
    C1 = ibfm.sampled_circle(2,3,choice_C.count(1), RandGen)+[0,4.5]
    C2 = ibfm.sampled_circle(2,3,choice_C.count(2), RandGen)-[2,0]
    return np.vstack((C0, C1, C2, uniform))

In [ ]:
RandGen = np.random.default_rng(5)
X = three_circles_distribution(0.8, 120, RandGen)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,3))
ax.scatter(X[:,0], X[:,1], s=10, c="black")
plt.savefig("plots/bootstrap/sample_circles.png")

We have drawn a sample $X$ from a distribution around three circles.
Now we take another 10 samples from the same distribution and compute the prevalence of cycles. For each sample Y, there is a sequence $X \hookrightarrow Z \hookleftarrow Y$ where $Z=X\cup Y$.

In [ ]:
%%capture
output_dir="output/"
Dist_X = dist.squareform(dist.pdist(X))
IBloFunMatch_X = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_X, range(len(X)), output_dir)
X_barcode = IBloFunMatch_X["X_barcode_1"]
X_reps = IBloFunMatch_X["X_reps_1"]

In [ ]:
def compute_prevalence(Dist_X, Dist_Y, Dist_Z):
    IBloFunMatch_XYZ = [
        ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, range(len(X)), output_dir),
        ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, range(len(X), len(Z)), output_dir)
    ]
    common_entries = set.intersection(set(IBloFunMatch_XYZ[0]["induced_matching_1"]), set(IBloFunMatch_XYZ[1]["induced_matching_1"]))
    common_entries = np.array(list(common_entries))
    common_entries = list(common_entries[common_entries>=0])
    affinities_common = np.zeros(IBloFunMatch_XYZ[0]["S_barcode_1"].shape[0])
    for idx in common_entries:
        gamma = IBloFunMatch_XYZ[0]["S_barcode_1"][IBloFunMatch_XYZ[0]["induced_matching_1"].index(idx)]
        im_bar = IBloFunMatch_XYZ[0]["X_barcode_1"][idx]
        delta = IBloFunMatch_XYZ[1]["S_barcode_1"][IBloFunMatch_XYZ[1]["induced_matching_1"].index(idx)]
        C_gamdel = (min(gamma[1], delta[1]) - max(gamma[0], delta[0]))/(max(gamma[1], delta[1])- min(gamma[0], delta[0]))
        a_gamma = (im_bar[1]-gamma[0])/(gamma[1]-gamma[0])
        a_delta = (im_bar[1]-delta[0])/(delta[1]-delta[0])
        affinity = a_gamma*a_delta*C_gamdel
        affinities_common[IBloFunMatch_XYZ[0]["induced_matching_1"].index(idx)] = a_gamma*a_delta*C_gamdel
    # end for 
    return list(affinities_common)

In [ ]:
%%capture
RandGen = np.random.default_rng(6)
num_resamples = 7
Dist_X = dist.squareform(dist.pdist(X))
affinities_resamples = []
for it_sample in range(num_resamples):
    Y = three_circles_distribution(0.8, 120, RandGen)
    Z = np.vstack((X, Y))
    Dist_Y = dist.squareform(dist.pdist(Y))
    Dist_Z = dist.squareform(dist.pdist(Z))
    affinities_resamples.append(compute_prevalence(Dist_X, Dist_Y, Dist_Z))

In [ ]:
affinities_arr = np.vstack(affinities_resamples)
affinities_mean = affinities_arr.mean(axis=0)
affinities_mean

In [ ]:
affinities_mean_sorted = affinities_mean[np.argsort(-affinities_mean)]
fig = plt.figure(figsize=(4,2))
plt.bar(range(len(affinities_mean_sorted)), affinities_mean_sorted, color="blue", width=0.6)
ax = plt.gca()
ax.set_ylim(0,1)
plt.savefig("plots/bootstrap/affinities_three_circles.png")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(3,3))
ax.scatter(X[:,0], X[:,1], s=10, c="black")
for idx, affinity in enumerate(affinities_mean):
    cycle_rep = list(IBloFunMatch_X["S_reps_1"][idx])
    while len(cycle_rep)>0:
        edge = X[[cycle_rep.pop(), cycle_rep.pop()]]
        ax.plot(edge[:,0], edge[:,1], linewidth=3, c="red", alpha=affinity)
plt.savefig("plots/bootstrap/sample_circles_reps.png")

## Example basic Unstability
Simple example showing that matchigs are unstable generally.

In [ ]:
import gudhi

In [ ]:
np.linspace(0,1,10)

In [ ]:
def perfect_circle(r, n):
    angles = np.linspace(0,2*np.pi,n)
    return np.vstack((np.cos(angles)*r, np.sin(angles)*r)).transpose()

In [ ]:
C0 = perfect_circle(1, 15)[:-1]
C1 = perfect_circle(1.9, 29)[:-1]
X2 = np.vstack([C0,C1,[0,0]])
Y2_indices = list(range(0,len(C0), 2)) + list(range(len(C0), len(C0)+len(C1), 2)) + [len(C0)+len(C1)]
Y2 = X2[Y2_indices]
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(8,4))
ax[0].scatter(Y2[:,0], Y2[:,1], s=10, c="black")
ax[1].scatter(X2[:,0], X2[:,1], s=10, c="black")
plt.savefig("plots/concentric_circles/basic_unstability_pts.png")

In [ ]:
%%capture
output_dir="output/"
Dist_X2 = dist.squareform(dist.pdist(X2))
Dist_Y2 = dist.squareform(dist.pdist(Y2))
IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_Y2, Dist_X2, Y2_indices, output_dir)

In [ ]:
IBloFunMatch_o["pm_matrix_1"]

In [ ]:
IBloFunMatch_o["induced_matching_1"]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ibfm.plot_matching(IBloFunMatch_o, output_dir, ax, fig)
plt.savefig("plots/concentric_circles/barcode_X2_match.png")

Add noise to X2 and see how the matching is affected

In [ ]:
def sample_from_disk(r, RandGen):
    while True:
        point = np.array([RandGen.uniform(-r,r), RandGen.uniform(-r,r)])
        if np.sum(point**2)<r:
            return point

In [ ]:
%%capture
RandGen = np.random.default_rng(10)
eps=0.01
X2_n = np.copy(X2)
for row in range(X2_n.shape[0]):
    X2_n[row] = X2_n[row] + sample_from_disk(eps, RandGen)
Y2_n = X2_n[Y2_indices]
Dist_X2_n = dist.squareform(dist.pdist(X2_n))
Dist_Y2_n = dist.squareform(dist.pdist(Y2_n))
IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_Y2_n, Dist_X2_n, Y2_indices, output_dir)

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.scatter(X2[:,0], X2[:,1], s=5, color="black")
plt.scatter(X2_n[:,0], X2_n[:,1], s=5, color="blue")
plt.savefig("plots/concentric_circles/points_X2_mod.png")

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(7,3))
ibfm.plot_matching(IBloFunMatch_o, output_dir, ax, fig)
plt.savefig("plots/concentric_circles/barcode_X2_n_match.png")

In [ ]:
%%capture
RandGen = np.random.default_rng(10)
for idx in range(10):
    eps = 0.01+0.01*idx
    X2_n = np.copy(X2)
    for row in range(X2_n.shape[0]):
        X2_n[row] = X2_n[row] + sample_from_disk(eps, RandGen)
    Y2_n = X2_n[Y2_indices]
    Dist_X2_n = dist.squareform(dist.pdist(X2_n))
    Dist_Y2_n = dist.squareform(dist.pdist(Y2_n))
    IBloFunMatch_o = ibfm.get_IBloFunMatch_subset(Dist_Y2_n, Dist_X2_n, Y2_indices, output_dir)
    # Plot Points
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(10,10))
    ax[1][0].scatter(Y2_n[:,0], Y2_n[:,1], s=5, color="orange")
    ax[1][1].scatter(X2[:,0], X2[:,1], s=5, color="black")
    ax[1][1].scatter(X2_n[:,0], X2_n[:,1], s=5, color="blue")
    eps+=0.01
    # Plot Matching Barcodes
    ibfm.plot_matching(IBloFunMatch_o, output_dir, ax[0], fig)
    plt.savefig(f"plots/concentric_circles/matching_X2_n_{idx}.png")